## 1. Setup - Colab & Environment

In [ ]:
import sys
import os

# Colab kontrolü
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("✅ Google Colab ortamı tespit edildi")

    # GPU kontrolü
    import torch
    if torch.cuda.is_available():
        print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
        print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    else:
        print("⚠️  GPU bulunamadı! Runtime > Change runtime type > GPU seçin")

    # GitHub'dan projeyi klonla
    print("\n📥 Proje indiriliyor...")
    !git clone https://github.com/Aliekinozcetin/Mitre_Attack_TTP_Mapping.git

    # Proje dizinine geç
    os.chdir('Mitre_Attack_TTP_Mapping')
    print(f"✅ Çalışma dizini: {os.getcwd()}")

    # Gerekli paketleri yükle
    print("\n📦 Paketler yükleniyor...")
    !pip install -q torch transformers datasets scikit-learn pandas tqdm matplotlib seaborn
    print("✅ Tüm paketler yüklendi")
else:
    print("ℹ️  Yerel ortamda çalışıyorsunuz")

## 2. Import Modules

In [ ]:
import torch
import numpy as np
import json
from datetime import datetime
import pandas as pd
from pathlib import Path

from src.data_loader import prepare_data
from src.model import load_model
from src.train import train_model
from src.evaluate import evaluate_model
from src.strategies import get_strategy_config

print("✅ Modüller yüklendi")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 3. Configuration - Strategy Selection

In [ ]:
# Base configuration
BASE_CONFIG = {
    'model_name': 'bert-base-uncased',
    'max_length': 512,
    'batch_size': 16,
    'learning_rate': 2e-5,
    'num_epochs': 3,  # Her strateji için 3 epoch
    'warmup_steps': 500,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'output_dir': './outputs_comparison'
}

# 🎯 HANGİ STRATEJİLERİ TEST ETMEK İSTİYORSUNUZ?
# True = Çalıştır, False = Atla
STRATEGIES_TO_RUN = {
    'baseline': True,        # Referans için mutlaka çalıştırın
    'focal_weak': True,      # Standart Focal Loss
    'focal_strong': True,    # Agresif Focal Loss
    'weighted': True,        # Class Weights (en umut verici)
    'oversampled': False,    # Oversampling (yavaş olabilir)
    'subset_50': False,      # Top-50 etiket (hızlı test için)
    'subset_100': True,      # Top-100 etiket (dengeli)
}

print("\n" + "="*70)
print("🔬 COMPARISON PIPELINE CONFIGURATION")
print("="*70)
print(f"Model: {BASE_CONFIG['model_name']}")
print(f"Epochs: {BASE_CONFIG['num_epochs']}")
print(f"Device: {BASE_CONFIG['device']}")
print(f"\nAktif stratejiler ({sum(STRATEGIES_TO_RUN.values())} adet):")
for strategy, enabled in STRATEGIES_TO_RUN.items():
    status = "✅" if enabled else "⏭️ "
    print(f"  {status} {strategy}")
print("="*70)

## 4. Data Preparation

In [ ]:
print("\n" + "="*70)
print("STEP 1: DATA PREPARATION")
print("="*70 + "\n")

data = prepare_data(
    model_name=BASE_CONFIG['model_name'],
    max_length=BASE_CONFIG['max_length']
)

train_dataset = data['train_dataset']
test_dataset = data['test_dataset']
label_list = data['label_list']
num_labels = data['num_labels']

print(f"\n✅ Veri hazırlama tamamlandı!")
print(f"   Train samples: {len(train_dataset)}")
print(f"   Test samples: {len(test_dataset)}")
print(f"   Number of labels: {num_labels}")

# Class imbalance analizi
all_train_labels = torch.stack([train_dataset[i]['labels'] for i in range(len(train_dataset))])
positive_ratio = all_train_labels.float().mean().item()
print(f"\n⚠️  Class Imbalance:")
print(f"   Positive label ratio: {positive_ratio:.4f} ({positive_ratio*100:.2f}%)")
print(f"   Imbalance ratio: 1:{int(1/positive_ratio)}")

## 5. Training Pipeline - Run All Strategies

In [ ]:
print("\n" + "="*70)
print("🔬 STARTING COMPREHENSIVE COMPARISON")
print("="*70)

# Results storage
all_results = []
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create main output directory
main_output_dir = Path(BASE_CONFIG['output_dir']) / f"comparison_{timestamp}"
main_output_dir.mkdir(parents=True, exist_ok=True)

print(f"\n📁 Main output directory: {main_output_dir}\n")

# Iterate through strategies
for strategy_name, enabled in STRATEGIES_TO_RUN.items():
    if not enabled:
        print(f"\n⏭️  Skipping {strategy_name}")
        continue
    
    print("\n" + "="*80)
    print(f"🎯 TRAINING STRATEGY: {strategy_name.upper()}")
    print("="*80)
    
    try:
        # Get strategy configuration
        strategy_config = get_strategy_config(
            strategy_name, 
            train_dataset, 
            num_labels, 
            label_list
        )
        
        print(f"\n📋 Strategy: {strategy_config['name']}")
        print(f"   Description: {strategy_config['description']}")
        print(f"   Num labels: {strategy_config['num_labels']}")
        
        # Create strategy output directory
        strategy_output_dir = main_output_dir / strategy_name
        strategy_output_dir.mkdir(exist_ok=True)
        
        # Load model with strategy-specific settings
        model = load_model(
            model_name=BASE_CONFIG['model_name'],
            num_labels=strategy_config['num_labels'],
            device=BASE_CONFIG['device'],
            use_focal_loss=strategy_config.get('use_focal_loss', False),
            focal_alpha=strategy_config.get('focal_alpha', 0.25),
            focal_gamma=strategy_config.get('focal_gamma', 2.0),
            pos_weight=strategy_config.get('pos_weight', None)
        )
        
        # Train model
        print(f"\n🚀 Starting training...")
        start_time = datetime.now()
        
        # Use custom dataloader if provided
        if strategy_config.get('custom_dataloader') is not None:
            # Custom training loop would go here
            # For now, use standard training
            pass
        
        history = train_model(
            model=model,
            train_dataset=strategy_config['dataset'],
            test_dataset=test_dataset,
            output_dir=str(strategy_output_dir),
            batch_size=BASE_CONFIG['batch_size'],
            learning_rate=BASE_CONFIG['learning_rate'],
            num_epochs=BASE_CONFIG['num_epochs'],
            warmup_steps=BASE_CONFIG['warmup_steps'],
            device=BASE_CONFIG['device']
        )
        
        training_time = (datetime.now() - start_time).total_seconds() / 60
        print(f"\n⏱️  Training completed in {training_time:.1f} minutes")
        
        # Evaluate model
        print(f"\n📊 Evaluating...")
        
        # Test different thresholds
        best_f1 = 0
        best_threshold = 0.5
        best_metrics = None
        
        for thresh in [0.1, 0.2, 0.3, 0.4, 0.5]:
            metrics = evaluate_model(
                model=model,
                test_dataset=test_dataset,
                batch_size=BASE_CONFIG['batch_size'],
                device=BASE_CONFIG['device'],
                threshold=thresh,
                label_list=strategy_config['label_list']
            )
            
            if metrics['micro_f1'] > best_f1:
                best_f1 = metrics['micro_f1']
                best_threshold = thresh
                best_metrics = metrics
        
        # Top-K evaluation
        print(f"\n🔬 Top-K evaluation...")
        model.eval()
        topk_results = []
        
        for k in [1, 3, 5, 10]:
            all_predictions = []
            all_labels = []
            
            test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)
            
            with torch.no_grad():
                for batch in test_loader:
                    outputs = model(
                        input_ids=batch['input_ids'].to(BASE_CONFIG['device']),
                        attention_mask=batch['attention_mask'].to(BASE_CONFIG['device'])
                    )
                    probs = torch.sigmoid(outputs['logits'])
                    
                    batch_preds = torch.zeros_like(probs)
                    topk_values, topk_indices = torch.topk(probs, k=min(k, probs.size(1)), dim=1)
                    batch_preds.scatter_(1, topk_indices, 1.0)
                    
                    all_predictions.append(batch_preds.cpu().numpy())
                    all_labels.append(batch['labels'].cpu().numpy())
            
            predictions = np.vstack(all_predictions)
            labels = np.vstack(all_labels)
            
            from sklearn.metrics import f1_score, precision_score, recall_score
            
            topk_f1 = f1_score(labels, predictions, average='micro', zero_division=0)
            topk_results.append({'k': k, 'micro_f1': topk_f1})
        
        best_topk = max(topk_results, key=lambda x: x['micro_f1'])
        
        # Store results
        result = {
            'strategy': strategy_name,
            'strategy_name': strategy_config['name'],
            'description': strategy_config['description'],
            'num_labels': strategy_config['num_labels'],
            'training_time_min': training_time,
            'final_loss': history['train_loss'][-1],
            'best_threshold': best_threshold,
            'threshold_micro_f1': best_metrics['micro_f1'],
            'threshold_macro_f1': best_metrics['macro_f1'],
            'threshold_samples_f1': best_metrics['samples_f1'],
            'best_topk_k': best_topk['k'],
            'topk_micro_f1': best_topk['micro_f1'],
        }
        
        all_results.append(result)
        
        # Save strategy results
        with open(strategy_output_dir / 'results.json', 'w') as f:
            json.dump(result, f, indent=2)
        
        with open(strategy_output_dir / 'training_history.json', 'w') as f:
            json.dump(history, f, indent=2)
        
        print(f"\n✅ {strategy_name} completed!")
        print(f"   Best Threshold F1: {result['threshold_micro_f1']:.4f} @ {result['best_threshold']}")
        print(f"   Best Top-K F1: {result['topk_micro_f1']:.4f} @ K={result['best_topk_k']}")
        
        # Clean up model to free memory
        del model
        torch.cuda.empty_cache()
        
    except Exception as e:
        print(f"\n❌ Error in {strategy_name}: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

print("\n" + "="*80)
print("✅ ALL STRATEGIES COMPLETED!")
print("="*80)

## 6. Results Comparison & Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

print("\n" + "="*80)
print("📊 COMPREHENSIVE RESULTS COMPARISON")
print("="*80)

# Create results DataFrame
df_results = pd.DataFrame(all_results)

# Display results table
print("\n📋 RESULTS TABLE:")
print("="*80)
display_cols = ['strategy_name', 'num_labels', 'training_time_min', 
                'threshold_micro_f1', 'topk_micro_f1', 'best_topk_k']
print(df_results[display_cols].to_string(index=False))
print("="*80)

# Find best strategies
best_threshold = df_results.loc[df_results['threshold_micro_f1'].idxmax()]
best_topk = df_results.loc[df_results['topk_micro_f1'].idxmax()]

print(f"\n🏆 BEST STRATEGIES:")
print(f"\nThreshold-based:")
print(f"  Strategy: {best_threshold['strategy_name']}")
print(f"  Micro F1: {best_threshold['threshold_micro_f1']:.4f}")
print(f"  Threshold: {best_threshold['best_threshold']}")

print(f"\nTop-K based:")
print(f"  Strategy: {best_topk['strategy_name']}")
print(f"  Micro F1: {best_topk['topk_micro_f1']:.4f}")
print(f"  K: {best_topk['best_topk_k']}")

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. F1 Score Comparison
ax = axes[0, 0]
x = np.arange(len(df_results))
width = 0.35
ax.bar(x - width/2, df_results['threshold_micro_f1'], width, label='Threshold-based', alpha=0.8)
ax.bar(x + width/2, df_results['topk_micro_f1'], width, label='Top-K based', alpha=0.8)
ax.set_xlabel('Strategy')
ax.set_ylabel('Micro F1 Score')
ax.set_title('F1 Score Comparison Across Strategies')
ax.set_xticks(x)
ax.set_xticklabels(df_results['strategy'], rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Training Time
ax = axes[0, 1]
ax.bar(df_results['strategy'], df_results['training_time_min'], alpha=0.8, color='coral')
ax.set_xlabel('Strategy')
ax.set_ylabel('Training Time (minutes)')
ax.set_title('Training Time Comparison')
ax.set_xticklabels(df_results['strategy'], rotation=45, ha='right')
ax.grid(True, alpha=0.3)

# 3. F1 vs Training Time (Efficiency)
ax = axes[1, 0]
scatter = ax.scatter(df_results['training_time_min'], df_results['topk_micro_f1'], 
                     s=200, alpha=0.6, c=range(len(df_results)), cmap='viridis')
for idx, row in df_results.iterrows():
    ax.annotate(row['strategy'], 
                (row['training_time_min'], row['topk_micro_f1']),
                xytext=(5, 5), textcoords='offset points', fontsize=8)
ax.set_xlabel('Training Time (minutes)')
ax.set_ylabel('Top-K Micro F1')
ax.set_title('Efficiency: F1 Score vs Training Time')
ax.grid(True, alpha=0.3)

# 4. Macro F1 Comparison (for rare labels)
ax = axes[1, 1]
ax.bar(df_results['strategy'], df_results['threshold_macro_f1'], alpha=0.8, color='lightgreen')
ax.set_xlabel('Strategy')
ax.set_ylabel('Macro F1 Score')
ax.set_title('Macro F1 (Rare Label Performance)')
ax.set_xticklabels(df_results['strategy'], rotation=45, ha='right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(main_output_dir / 'comparison_results.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Visualization saved: comparison_results.png")

# Save results CSV
df_results.to_csv(main_output_dir / 'comparison_results.csv', index=False)
print(f"📊 Results saved: comparison_results.csv")

## 7. Detailed Analysis & Recommendations

In [ ]:
print("\n" + "="*80)
print("🔍 DETAILED ANALYSIS & RECOMMENDATIONS")
print("="*80)

# Calculate improvements over baseline
if 'baseline' in df_results['strategy'].values:
    baseline_f1 = df_results[df_results['strategy'] == 'baseline']['topk_micro_f1'].values[0]
    
    print(f"\n📊 Improvement over baseline (F1={baseline_f1:.4f}):")
    print("-" * 80)
    
    for idx, row in df_results.iterrows():
        if row['strategy'] == 'baseline':
            continue
        
        improvement = ((row['topk_micro_f1'] - baseline_f1) / baseline_f1) * 100
        symbol = "📈" if improvement > 0 else "📉"
        
        print(f"{symbol} {row['strategy_name']:30s}: {row['topk_micro_f1']:.4f} ({improvement:+.1f}%)")

# Recommendations
print(f"\n\n💡 RECOMMENDATIONS:")
print("="*80)

# Best overall
best_overall = df_results.loc[df_results['topk_micro_f1'].idxmax()]
print(f"\n1️⃣ EN İYİ GENEL PERFORMANS:")
print(f"   ✅ {best_overall['strategy_name']}")
print(f"   F1 Score: {best_overall['topk_micro_f1']:.4f}")
print(f"   Training Time: {best_overall['training_time_min']:.1f} min")

# Best efficiency (F1 per minute)
df_results['efficiency'] = df_results['topk_micro_f1'] / df_results['training_time_min']
best_efficient = df_results.loc[df_results['efficiency'].idxmax()]
print(f"\n2️⃣ EN VERİMLİ YAKLAŞIM:")
print(f"   ⚡ {best_efficient['strategy_name']}")
print(f"   F1/min: {best_efficient['efficiency']:.6f}")

# Best for rare labels
best_rare = df_results.loc[df_results['threshold_macro_f1'].idxmax()]
print(f"\n3️⃣ NADİR ETİKETLER İÇİN EN İYİ:")
print(f"   🎯 {best_rare['strategy_name']}")
print(f"   Macro F1: {best_rare['threshold_macro_f1']:.4f}")

print("\n" + "="*80)

## 8. Download Results

In [ ]:
if IN_COLAB:
    import shutil
    from google.colab import files

    # Sonuçları ZIP'le
    zip_name = f"comparison_results_{timestamp}"
    shutil.make_archive(zip_name, 'zip', main_output_dir)

    print(f"\n📦 Sonuçlar sıkıştırılıyor: {zip_name}.zip")
    print(f"   Boyut: {os.path.getsize(zip_name + '.zip') / (1024*1024):.2f} MB")

    # İndir
    print("\n⬇️  İndirme başlatılıyor...")
    files.download(zip_name + '.zip')
    print("✅ İndirme tamamlandı!")
else:
    print(f"ℹ️  Yerel ortamdasınız, sonuçlar: {main_output_dir}")

## 📋 Summary

Bu notebook ile:

1. ✅ 7 farklı class imbalance stratejisi test edildi
2. ✅ Her strateji için F1, Precision, Recall hesaplandı
3. ✅ Training time ve efficiency karşılaştırıldı
4. ✅ Threshold ve Top-K yaklaşımları denendi
5. ✅ Görselleştirmeler oluşturuldu
6. ✅ En iyi stratejiler belirlendi

**Sonuçlar:** `outputs_comparison/comparison_YYYYMMDD_HHMMSS/`

**Öneriler:** Yukarıdaki "Detailed Analysis" bölümüne bakın.